<a href="https://colab.research.google.com/github/olyviah82/compilerconstruction/blob/main/A_labactivity5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

121455 Olyviah Nyaboke 
120174 Cynthia Gichuki 
121759 Shamsa Abdullahi 

Review what syntax analysis is and in a few words, describe what this phase of compiler construction entails




> Syntax Analysis also known as parsing: In every programming lannguage has precise rules that prescribes the syntactic structure of well-formed programs.


>Syntax Analysis:  It checks the syntactical structure of the given input, i.e. whether the given input is in the correct syntax (of the language in which the input has been written) or not.

> It is the second phase of the compiler after the lexical analysis. 
The parser uses the first components of the tokens produced by the lexical analyzer to create a tree-like intermediate representation that depicts the grammatical structure of the token stream.

> The syntax tree is constructed using the pre-defined grammar of the language and the input string.If the input string can be produced with the help of the synatx tree in the derivation process,the input string is found to be in the correct syntax, if not the error is reported by the synatax analyzer.

> In this phase, there is a generation of a syntax tree in which each interior node represents an operation and the children of the node represents an operation.

Using Flex (lexical analyzer) and Bison (yacc), show how one can develop a simple
syntax analyzer. [Consult the web, show and explain the simple codes that you find].

Flex is a scanner generator tool for lexical analysis, which is based on finite
state machine (FSM). The input is a set of regular expressions, and the
output is the code to implement the scanner according to the input rules.
To implement a scanner for calculator, we can write the file “cal1.l” as
below:

/* this is only for scanner, not link with parser yet */
%{
int lineNum = 0;
%}
%%
"(" { printf("(\n"); }
")" { printf(")\n"); }
"+" { printf("+\n"); }
"*" { printf("*\n"); }
\n { lineNum++; }
[ \t]+ { }
[0-9]+ { printf("%s\n", yytext); }
%%
int yywrap() {
 return 1;
}
int main () {
 yylex();
 return 0;
} 

Here is the Makefile used to build the scanner:
p1: lex.yy.o
 gcc -g -o p1 lex.yy.o
lex.yy.o: cal1.l
 flex cal1.l; gcc -g -c lex.yy.c
clean:
 rm -f p1 *.o lex.yy.c
 

Bison is a LALR(1) parser generator tool for syntax analysis, which is based
on pushdown automata (PDA). The input is a set of context-free grammar
(CFG) rules, and the output is the code to implement the parser according to
the input rules.
To implement a parser for calculator, we can write the file “cal.y” as below:
%{
 #include <stdio.h>
#include <ctype.h>
int lineNum = 1;
void yyerror(char *ps, ...) { /* need this to avoid
link problem */
 printf("%s\n", ps);
}
%}
%union {
 int d;
}
// need to choose token type from union above
%token <d> NUMBER
%token '(' ')'
%left '+'
%left '*'
%type <d> exp factor term
%start cal
%%
cal
 : exp
 { printf("The result is %d\n", $1); }
 ;
exp
 : exp '+' factor
 { $$ = $1 + $3; }
 | factor
 { $$ = $1; }
 ;
factor
 : factor '*' term
 { $$ = $1 * $3; }
 | term
 { $$ = $1; }
 ;
term
 : NUMBER
 { $$ = $1; }
 | '(' exp ')'
 { $$ = $2; }
 ;
%%
int main() {
 yyparse();
 return 0;
} 

To integrate both the scanner and parser, we need to modify the scanner
input file “cal1.l” and save it as “cal.l” as below:
%{
#include <stdlib.h> /* for atoi call */
#define DEBUG /* for debuging: print tokens and
their line numbers */
#define NUMBER 258 /* copy this from cal.tab.c */
typedef union { /* copy this from cal.tab.c */
 int d;
} YYSTYPE;
YYSTYPE yylval; /* for passing value to parser */
extern int lineNum; /* line number from cal.tab.c */
%}
%%
[ \t]+ {}
[\n] { lineNum++; }
"(" {
#ifdef DEBUG
 printf("token '(' at line %d\n", lineNum);
#endif
 return '(';
 }
")" {
#ifdef DEBUG
 printf("token ')' at line %d\n", lineNum);
#endif
 return ')';
 }
"+" {
#ifdef DEBUG
 printf("token '+' at line %d\n", lineNum);
#endif
 return '+';
 }
"*" {
#ifdef DEBUG
 printf("token '*' at line %d\n", lineNum);
#endif
 return '*';
 }
[0-9]+ {
#ifdef DEBUG
 printf("token %s at line %d\n", yytext, lineNum);
#endif
 yylval.d = atoi(yytext);
 return NUMBER;
}
%%
int yywrap() { /* need this to avoid link problem */
 return 1;
} 